In [1]:
# lib imports
from langchain_community.vectorstores import Chroma
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_community.embeddings import OllamaEmbeddings
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.retrievers.multi_query import MultiQueryRetriever

In [2]:
# load vector (saved from lg.py)
load_vector_db=Chroma(persist_directory="local_rag_vector_db",
                      embedding_function=OllamaEmbeddings(model="nomic-embed-text"),
                      collection_name="local_rag_vector_db")


In [3]:
# test if vector DB was loaded correctly by searching for a word 
docs = load_vector_db.similarity_search('Harwyn Hardhand')
docs

[Document(metadata={'page': 3, 'source': 'chap1_fire_and_blood.pdf'}, page_content='in Westeros long predated the events that drove him to war. As well,\nthere are reliable reports of Aegon and his sister Visenya visiting the\nCitadel of Oldtown in their youth, and hawking on the Arbor as guests\nof Lord Redwyne. He may have visited Lannisport as well; accounts\ndiffer.\nThe Westeros of Aegon’s youth was divided into seven quarrelsome\nkingdoms, and there was hardly a time when two or three of these\nkingdoms were not at war with one another. The vast, cold, stony\nNorth was ruled by the Starks of Winterfell. In the deserts of Dorne,\nthe Martell princes held sway. The gold-rich westerlands were ruled by\nthe Lannisters of Casterly Rock, the fertile Reach by the Gardeners of\nHighgarden. The Vale, the Fingers, and the Mountains of the Moon\nbelonged to House Arryn…but the most belligerent kings of Aegon’s\ntime were the two whose realms lay closest to Dragonstone, Harren\nthe Black and

In [4]:
!ollama list

NAME                   	ID          	SIZE  	MODIFIED     
nomic-embed-text:latest	0a109f422b47	274 MB	45 hours ago	
codellama:latest       	8fdf8f752f6e	3.8 GB	47 hours ago	
llama3:latest          	365c0bd3c000	4.7 GB	2 days ago  	
llama2:latest          	78e26419b446	3.8 GB	2 days ago  	


In [5]:
# retriever prompt
QUERY_PROMPT = PromptTemplate(
    input_variables=['question'],
    template = """You are an AI language model assistant.
        Your task is to answer questions about the chapters of a book.
        Also generate a short perspective about the context of your answer based on the chapter.
        Don't try to make up an answer, if you don't find the answer just say that you don't know.
        Question: {question}
        """)

# RAG prompt
template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""

In [6]:
model = 'codellama'
llm = ChatOllama(model=model)

# retriever
retriever = MultiQueryRetriever.from_llm(
    load_vector_db.as_retriever(), 
    llm,
    prompt=QUERY_PROMPT
)

# create chain
prompt = ChatPromptTemplate.from_template(template)
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [7]:
print(chain.invoke("Who is Aegon I Targaryen?"))

Aegon I Targaryen was the first king of House Targaryen. 


In [8]:
print(chain.invoke("What is the subject covered in this chapter?"))

 This chapter covers the events of Aegon's conquest of the Seven Kingdoms and how he became king after conquering the realm. 


In [9]:
model = 'llama3'
llm = ChatOllama(model=model)

# retriever
retriever = MultiQueryRetriever.from_llm(
    load_vector_db.as_retriever(), 
    llm,
    prompt=QUERY_PROMPT
)

# create chain
prompt = ChatPromptTemplate.from_template(template)
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [10]:
print(chain.invoke("Who is Aegon I Targaryen?"))

According to the provided context, Aegon I Targaryen is a king who kept his lands and titles after conquering the Seven Kingdoms. He is also referred to as "Aegon of House Targaryen".


In [11]:
print(chain.invoke("What is the subject covered in this chapter?"))

Based on the provided context, it appears that the subject covered in this chapter is the conquest of Westeros by Aegon Targaryen and his dragons. The chapter likely describes the events surrounding Aegon's invasion of Westeros, including his battles with various lords and kings, as well as the reactions of other kingdoms and factions to his claims of power.


In [12]:
load_vector_db.delete_collection()